# MOMA's datasets


- Pickled version of both artists and artworks datasets
- For the overall collection two pickle files, one for artworks created before 1980 and the other for later production
- FOr each MoMA's department two pickle files, one for artworks created before 1980 and the other for later production

In [2]:
#LIBRARIES
import re
import csv
import pandas as pd
import pandas_profiling as pp

In [3]:
#PATHS
path = './MoMA_data/'

### Pickle originals

Before pickling and further manipulating the data inside each set, we need to perform some preliminary cleaning of the columns. 

In [4]:
#ARTISTS
original_artists = pd.read_csv(path+'Artists.csv')
artists= original_artists[['ConstituentID','DisplayName','Nationality','Gender','BeginDate','EndDate','Wiki QID','ULAN']]

In [5]:
artists.rename(columns={'ConstituentID': 'ID', 'DisplayName': 'Artist','BeginDate': 'Birth', 'EndDate': 'Death'}, inplace=True)

artists['Gender'] = artists['Gender'].fillna('missing').astype('str')
artists['Wiki QID'] = artists['Wiki QID'].fillna('missing').astype('str')
artists['ULAN'] = artists['ULAN'].fillna('0').astype('int')
artists['ID'] = artists['ID'].fillna('0').astype('str')
artists['Artist'] = artists['Artist'].fillna('Unknown').astype('str')
artists['Nationality'] = artists['Nationality'].fillna('missing').astype('str')
artists['Gender'] = artists['Gender'].fillna('missing').astype('str')
artists['Birth'] = artists['Birth'].fillna('0').astype('int')
artists['Death'] = artists['Death'].fillna('0').astype('int')



#artists.to_pickle(f'./MoMA_data/artists.pkl')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/qj/xh48vw9d4z5_h6wzs6r2smdh0000gn/T/ipykernel_68476/192219770.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artists['Gender'] = artists['Gender'].fillna('missing').astype('str')
/var/folders/qj/xh48vw9d4z5_h6wzs6r2smdh0000gn/T/ipykernel_68476/192219770.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

Normalize Gender labels to 'M', 'F' and 'NB' to make them easier to compare with Rizome's data 

In [6]:
for x, row in artists.iterrows():
    if re.match(r'[M|m]ale', getattr(row, 'Gender')):
        artists.at[x, 'Gender'] = 'M'
                
    elif re.match(r'[F|f]emale',getattr(row, 'Gender')):
        artists.at[x, 'Gender'] = 'F'

    elif re.match(r'Non-[B|b]inary', getattr(row, 'Gender')):
        artists.at[x, 'Gender'] = 'NB'

In [7]:
artists.to_pickle(f'./MoMA_data/pickle/MoMAartists.pkl')

In [8]:
#ARTWORKS
original_artworks = pd.read_csv(path+'Artworks.csv')
original_artworks =  original_artworks[['Title','Artist','ConstituentID','Date','Medium','Department','DateAcquired','URL','ThumbnailURL']]
original_artworks.to_pickle(f'./MoMA_data/pickle/MoMAartworks.pkl')

### Divide artworks into two datasets

The original artworks set is split into two datasets, throughout this little algoritm.

!! COMPUTATIONAL TIME OVER 20min!!

In [9]:
#SET THE TWO NEW FRAMES
before80 = pd.DataFrame(columns =['Title','Artist','ConstituentID','Date','Medium','Department','DateAcquired','URL','ThumbnailURL'])
after80 = pd.DataFrame(columns =['Title','Artist','ConstituentID','Date','Medium','Department','DateAcquired','URL','ThumbnailURL'])

#START FUNCTION
for index, row in original_artworks.iterrows():

    if  re.match('.*\d\d\d\d.*', str(row[3])):
        year = re.findall(r'\d\d\d\d', row[3])
        row[3] = str(year[0])
        if int(row[3]) < 1980:
            before80 = before80.append([row])
        else:
            after80 = after80.append([row])
    
    

    

After the creation of before and after 1980 subsets we looked for missing values and normalized them with the strings 'Unknown' or 'missing' depending on the kind of data. Additionally we used the .astype command, to meke sure pandas parses/recognizes them using their correct data type.
Finally, we pickle the subsets, saving them in the /pickle/ directory 

In [12]:
before80

,Title,Artist,ID,DateCreated,Medium,Department,DateAcquired,URL,ThumbnailURL
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,1896,Ink and cut-and-pasted painted pages on paper,Architecture & Design,1996-04-09,http://www.moma.org/collection/works/2,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,1903,"Graphite, pen, color pencil, ink, and gouache ...",Architecture & Design,1997-01-15,http://www.moma.org/collection/works/4,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,1903,"Graphite, color pencil, ink, and gouache on tr...",Architecture & Design,1997-01-15,http://www.moma.org/collection/works/6,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...
5,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1976,Gelatin silver photograph,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/7,http://www.moma.org/media/W1siZiIsIjE0OCJdLFsi...
6,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1976,Gelatin silver photographs,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/8,http://www.moma.org/media/W1siZiIsIjE0OSJdLFsi...
...,...,...,...,...,...,...,...,...,...
138146,Untitled,"Chesnutt Brothers Studio, Andrew Chesnutt, Lew...","133005, 133006, 133007",1890,Gelatin silver print,Photography,2020-10-07,http://www.moma.org/collection/works/418928,http://www.moma.org/media/W1siZiIsIjQ5MjcyMiJd...
138147,Plate (folio 2 verso) from Muscheln und schirm...,Sophie Taeuber-Arp,5777,1939,One from an illustrated book with four line bl...,Drawings & Prints,2019-12-11,http://www.moma.org/collection/works/419286,http://www.moma.org/media/W1siZiIsIjQ4NTExNSJd...
138148,Plate (folio 6) from Muscheln und schirme (She...,Sophie Taeuber-Arp,5777,1939,One from an illustrated book with four line bl...,Drawings & Prints,2019-12-11,http://www.moma.org/collection/works/419287,http://www.moma.org/media/W1siZiIsIjQ4NTExOCJd...
138149,Plate (folio 12) from Muscheln und schirme (Sh...,Sophie Taeuber-Arp,5777,1939,One from an illustrated book with four line bl...,Drawings & Prints,2019-12-11,http://www.moma.org/collection/works/419288,http://www.moma.org/media/W1siZiIsIjQ4NTEyMCJd...


In [14]:
before80['Title'] = before80['Title'].fillna('Unknown').astype('str')
before80['Artist'] = before80['Artist'].fillna('Unknown').astype('str')
before80['ConstituentID'] = before80['ConstituentID'].fillna('missing').astype('str')
before80['Medium'] = before80['Medium'].fillna('missing').astype('str')
before80['Date'] = before80['Date'].fillna('0').astype('int')
before80['URL'] = before80['URL'].fillna('missing').astype('str')
before80['ThumbnailURL'] = before80['ThumbnailURL'].fillna('missing').astype('str')

before80.rename(columns={'ConstituentID': 'ID', 'Date': 'DateCreated'}, inplace=True)

KeyError: 'ConstituentID'

In [15]:
after80['Title'] = after80['Title'].fillna('Unknown').astype('str')
after80['Artist'] = after80['Artist'].fillna('Unknown').astype('str')
after80['ConstituentID'] = after80['ConstituentID'].fillna('missing').astype('str')
after80['Medium'] = after80['Medium'].fillna('missing').astype('str')
after80['Date'] = after80['Date'].fillna('0').astype('int')
after80['URL'] = after80['URL'].fillna('missing').astype('str')
after80['ThumbnailURL'] = after80['ThumbnailURL'].fillna('missing').astype('str')

after80.rename(columns={'ConstituentID': 'ID', 'Date': 'DateCreated'}, inplace=True)

In [16]:
after80

,Title,Artist,ID,DateCreated,Medium,Department,DateAcquired,URL,ThumbnailURL
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,1987,Paint and colored pencil on print,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/3,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Photographic reproduction with colored synthet...,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/5,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...
31,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Photographic reproduction with colored synthet...,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/33,http://www.moma.org/media/W1siZiIsIjIwMCJdLFsi...
35,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Photographic reproduction with colored synthet...,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/38,http://www.moma.org/media/W1siZiIsIjI2NyJdLFsi...
40,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Ink on tracing paper,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/44,http://www.moma.org/media/W1siZiIsIjI5NiJdLFsi...
...,...,...,...,...,...,...,...,...,...
138114,Cóctel (Cocktail),Alejandro Kuropatwa,132939,1996,Chromogenic color print,Photography,2020-10-07,missing,missing
138115,Cóctel (Cocktail),Alejandro Kuropatwa,132939,1996,Chromogenic color print,Photography,2020-10-07,missing,missing
138116,Cóctel (Cocktail),Alejandro Kuropatwa,132939,1996,Chromogenic color print,Photography,2020-10-07,missing,missing
138117,Cóctel (Cocktail),Alejandro Kuropatwa,132939,1996,Chromogenic color print,Photography,2020-10-07,missing,missing


## Artists and nationality for old and modern MOMA

Function performing the task for both datasates

In [17]:
def get_gender_nat(df, artists):
    for index, i in df.iterrows():    
        if ', 'in i['ID']:
            IDs = i['ID'].split(', ')
            genders = list()
            nationalities = list()
            for x in IDs:
                 gender = getattr(artists[artists['ID']== x], 'Gender')
                 gender = gender.values
                 gender = gender[0]
                 genders.append(gender)
                 nationality = getattr(artists[artists['ID']== x], 'Nationality')
                 nationality = nationality.values
                 nationality = nationality[0]
                 nationalities.append("".join(nationality))
            df.at[index, 'Gender'] = ", ".join(genders)
            df.at[index, 'Nationality'] = ", ".join(nationalities)    
        else:
            gender_s = getattr(artists[artists['ID'] == i['ID']], 'Gender')
            gender_s = gender_s.values
            nationality_s = getattr(artists[artists['ID'] == i['ID']], 'Nationality')
            nationality_s = nationality_s.values
            df.at[index, 'Gender'] = "".join(gender_s)
            df.at[index, 'Nationality'] = "".join(nationality_s)
            
    return df

In [18]:
# ADD GENDER AND NAT TO OLDER ARTWORKS
# VERY SLOW UP TO 6 MINS

before80['Nationality'] = "to do"
before80['Gender'] = "to do"
vizOldincorrect = get_gender_nat(before80, artists)
before80 = vizOldincorrect.replace([''], ['missing'], regex=True)

In [19]:
# ADD GENDER AND NAT TO NEWER ARTWORKS
# VERY SLOW UP TO 6 MINS

after80['Nationality'] = "to do"
after80['Gender'] = "to do"
vizNewincorrect = get_gender_nat(after80, artists)
after80 = vizNewincorrect.replace([''], ['missing'], regex=True)

In [20]:
after80.to_pickle(f'./MoMA_data/pickle/new_artworks.pkl')
before80.to_pickle(f'./MoMA_data/pickle/old_artworks.pkl')

## Pickle departments 

in order to make further analysis easier on MoMA's data we decided to additionally produce a subset for each department, divided by creation date as well as the two ones created in the precedent step  

In [75]:
#create pickles for each dempartment in before80 csv

architecture_design = before80[before80['Department'] == "Architecture & Design"]
architecture_design.to_pickle(f'./MoMA_data/pickle/departments/architecture_design_mod.pkl')

architecture_design_img  = before80[before80['Department'] == "Architecture & Design - Image Archive"]
architecture_design_img.to_pickle(f'./MoMA_data/pickle/departments/architecture_design_img_mod.pkl')

draws_prints= before80[before80['Department'] == "Drawings & Prints"]
draws_prints.to_pickle(f'./MoMA_data/pickle/departments/draws_prints_mod.pkl')

films= before80[before80['Department'] == "Film"]
films.to_pickle(f'./MoMA_data/pickle/departments/films_mod.pkl')

fluxus= before80[before80['Department'] == "Fluxus Collection"]
fluxus.to_pickle(f'./MoMA_data/pickle/departments/fluxus_mod.pkl')

media_perf= before80[before80['Department'] == "Media and Performance"]
media_perf.to_pickle(f'./MoMA_data/pickle/departments/media_perf_mod.pkl')

painting_sculp= before80[before80['Department'] == "Painting & Sculpture"]
painting_sculp.to_pickle(f'./MoMA_data/pickle/departments/paint_sculp_mod.pkl')

photo= before80[before80['Department'] == "Photography"]
photo.to_pickle(f'./MoMA_data/pickle/departments/photo_mod.pkl')

In [77]:
architecture_design = after80[after80['Department'] == "Architecture & Design"]
architecture_design.to_pickle(f'./MoMA_data/pickle/departments/architecture_design_cont.pkl')

architecture_design_img  = after80[after80['Department'] == "Architecture & Design - Image Archive"]
architecture_design_img.to_pickle(f'./MoMA_data/pickle/departments/architecture_design_img_cont.pkl')

draws_prints= after80[after80['Department'] == "Drawings & Prints"]
draws_prints.to_pickle(f'./MoMA_data/pickle/departments/draws_prints_cont.pkl')

films= after80[after80['Department'] == "Film"]
films.to_pickle(f'./MoMA_data/pickle/departments/films_cont.pkl')

fluxus= after80[after80['Department'] == "Fluxus Collection"]
fluxus.to_pickle(f'./MoMA_data/pickle/departments/fluxus_cont.pkl')

media_perf= after80[after80['Department'] == "Media and Performance"]
media_perf.to_pickle(f'./MoMA_data/pickle/departments/media_perf_cont.pkl')

painting_sculp= after80[after80['Department'] == "Painting & Sculpture"]
painting_sculp.to_pickle(f'./MoMA_data/pickle/departments/paint_sculp_cont.pkl')

photo= after80[after80['Department'] == "Photography"]
photo.to_pickle(f'./MoMA_data/pickle/departments/photo_cont.pkl')